In [1]:
import pandas as pd
import numpy as np

In [2]:
%ls

Decision Trees in Practice.ipynb
lending-club-data.csv
module-6-assignment-train-idx.json
module-6-assignment-validation-idx.json


In [3]:
loans = pd.read_csv('lending-club-data.csv')

/Users/tdx/anaconda2/envs/ds-env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: + 1 if x == 0 else -1)

loans = loans.drop(['bad_loans'], axis=1)

In [8]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [10]:
loans = loans[features + [target]]

In [12]:
loans.fillna('N/A', inplace=True)

In [13]:
loans

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1
5,E,36 months,RENT,9 years,1
6,F,60 months,OWN,4 years,-1
7,B,60 months,RENT,< 1 year,-1
8,C,60 months,OWN,5 years,1
9,B,36 months,OWN,10+ years,1


In [16]:
def one_hot_encoding(dataset, column, remove_categorical_column=False, inplace=False):
    result_dataset = dataset if inplace else dataset.copy()
    
    if column not in dataset.columns:
        return result_dataset
    
    all_categories = ([u for u in set(dataset[column].values)])
    all_categories.sort()
    
    for category in all_categories:
        new_name = "%s = %s" % (column, category)
        result_dataset[new_name] = (result_dataset[column] == category).apply(lambda u: 1 if u else 0)
    
    if remove_categorical_column:
        result_dataset = result_dataset.drop([column], axis=1, errors='ignore')
    
    return result_dataset

In [17]:
def one_hot_encoding_multiple_cols(dataset, columns, remove_categorical_column=False, inplace=False):
    result_dataset = dataset if inplace else dataset.copy()
    
    for column in columns:
        result_dataset = one_hot_encoding(result_dataset, column, remove_categorical_column=False, inplace=True)
    
    if remove_categorical_column:
        print (remove_categorical_column, len(result_dataset.columns))
        result_dataset.drop(columns, axis=1, inplace=True, errors='ignore')
        print (len(result_dataset.columns))
    
    # Since result_dataset lost reference to dataset, we need to assign new reference.
    if inplace: 
        dataset = result_dataset
        
    return result_dataset

In [19]:
one_hot_encoding_multiple_cols(loans, features,
                               remove_categorical_column=True,
                               inplace=True
                              )

True 30
26


,safe_loans,grade = A,grade = B,grade = C,grade = D,grade = E,grade = F,grade = G,term = 36 months,term = 60 months,...,emp_length = 2 years,emp_length = 3 years,emp_length = 4 years,emp_length = 5 years,emp_length = 6 years,emp_length = 7 years,emp_length = 8 years,emp_length = 9 years,emp_length = < 1 year,emp_length = N/A
0,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,-1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7,-1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
8,1,0,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
9,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
train_idx = pd.read_json('module-6-assignment-train-idx.json')[0]
valid_idx = pd.read_json('module-6-assignment-validation-idx.json')[0]

In [36]:
train_data = loans.iloc[train_idx]
validation_data = loans.iloc[valid_idx]

In [37]:
validation_data

,safe_loans,grade = A,grade = B,grade = C,grade = D,grade = E,grade = F,grade = G,term = 36 months,term = 60 months,...,emp_length = 2 years,emp_length = 3 years,emp_length = 4 years,emp_length = 5 years,emp_length = 6 years,emp_length = 7 years,emp_length = 8 years,emp_length = 9 years,emp_length = < 1 year,emp_length = N/A
24,-1,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
41,-1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
60,-1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
93,-1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
132,-1,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
160,-1,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,0
162,-1,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
175,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
184,-1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
270,-1,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [38]:
train_data

,safe_loans,grade = A,grade = B,grade = C,grade = D,grade = E,grade = F,grade = G,term = 36 months,term = 60 months,...,emp_length = 2 years,emp_length = 3 years,emp_length = 4 years,emp_length = 5 years,emp_length = 6 years,emp_length = 7 years,emp_length = 8 years,emp_length = 9 years,emp_length = < 1 year,emp_length = N/A
1,-1,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
6,-1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
7,-1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
10,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
12,-1,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
18,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
21,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
23,-1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
45,-1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
48,-1,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


## Eary stopping methods for decision trees

In this section, we will extend the binary tree implementation from the previous assignment in order to handle some early stopping conditions. Recall the 3 early stopping methods that were discussed in lecture:

- Reached a maximum depth. (set by parameter max_depth).
- Reached a minimum node size. (set by parameter min_node_size).
- Don't split if the gain in error reduction is too small. (set by parameter min_error_reduction).

For the rest of this assignment, we will refer to these three as early stopping conditions 1, 2, and 3.

### Early stopping condition 1: Maximum depth


In [24]:
def intermediate_node_num_mistakes(labels_in_node):
    total_node = len(labels_in_node)
    
    if (total_node == 0):
        return 0
    safe_loans = len(list(filter(lambda u: u == 1, labels_in_node)))
    risky_loans = total_node - safe_loans
    
    return min(risky_loans, safe_loans)

In [29]:
def reached_minimum_node_size(data, min_node_size):
    return len(data) <= min_node_size

### Quiz question: Given an intermediate node with 6 safe loans and 3 risky loans, if the min_node_size parameter is 10, what should the tree learning algorithm do next?



stop! because 6+3 < 10

In [30]:
def error_reduction(error_before_split, error_after_split):
    return error_before_split - error_after_split

### Quiz question: Assume an intermediate node has 6 safe loans and 3 risky loans. For each of 4 possible features to split on, the error reduction is 0.0, 0.05, 0.1, and 0.14, respectively. If the minimum gain in error reduction parameter is set to 0.2, what should the tree learning algorithm do next?



stop! (0.2 > max(0.0, 0.05, 0.1, 0.14)

In [25]:
def best_splitting_feature(data, features, target):
    target_values = data[target]
    best_feature = None
    best_error = 1
    
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        error = (left_mistakes + right_mistakes) / num_data_points
        
        if (error < best_error):
            best_error = error
            best_feature = feature
    
    print ("Best feature: %s, best error: %.2f"  % (best_feature, best_error))
    
    return best_feature

In [26]:
def create_leaf(target_values):
    leaf = {
        'splitting_feature': None,
        'left': None,
        'right': None,
        'is_leaf': True
    }
    
    num_ones = len(target_values[target_values == 1])
    num_minus_ones = len(target_values[target_values == -1])
    
    leaf['prediction'] = 1 if num_ones > num_minus_ones else -1
    
    return leaf

In [77]:
def decision_tree_create(data, features, target, current_depth = 0, 
                         max_depth = 10,
                         min_node_size = 1,
                         min_error_reduction=0.0):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print ("--------------------------------------------------------------------")
    print ("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    
    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values) == 0:  ## YOUR CODE HERE
        print ("Stopping condition 1 reached.")
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == []:   
        print ("Stopping condition 2 reached.")
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Early stoppiong condition 1: Reached max depth limit
    if current_depth >= max_depth:  
        print ("Early stopping condition 1 reached. Reached maximum depth")
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)
    
    # Early stopping condition 2: Reached the minimum node size.
    if reached_minimum_node_size(data, min_node_size):
        print ("Early stopping condition 2 reached. Reached minimum node size.")
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    splitting_feature = best_splitting_feature(data, remaining_features, target)

    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1] ## Your code here
    
    # Early stopping condition 3: Minimum error reduction
    # Calculate the error before splitting (number of misclassified examples
    # divided by the total number of examples)
    error_before_split = intermediate_node_num_mistakes(target_values) / float(len(data))
    
    # Calculate the error after splitting
    left_mistakes = intermediate_node_num_mistakes(left_split[target])
    right_mistakes = intermediate_node_num_mistakes(right_split[target])
    error_after_split = (left_mistakes + right_mistakes) / float(len(data))
    
    # If the error reduction is LESS THAN OR EQUAL TO min_error_reduction, return a leaf.
    if min_error_reduction >= 0.0 and (error_reduction(error_before_split, error_after_split) <= min_error_reduction):
        print ("Early stopping condition 3 reached. Minimum error reduction.")
        return create_leaf(data)
    
    remaining_features.remove(splitting_feature)
    print ("Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print ("Creating leaf node.")
        return create_leaf(right_split[target])
        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, 
                                     remaining_features,
                                     target, 
                                     current_depth=current_depth + 1,
                                     max_depth=max_depth,
                                     min_node_size=min_node_size,
                                     min_error_reduction=min_error_reduction)
    
    right_tree = decision_tree_create(right_split,
                                      remaining_features,
                                      target,
                                      current_depth=current_depth + 1,
                                      max_depth=max_depth,
                                      min_node_size=min_node_size,
                                      min_error_reduction=min_error_reduction)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [78]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print ("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate:
             print ("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

### Building a tree

In [79]:
max_depth = 6
min_node_size = 100
min_error_reduction = 0.0

In [80]:
train_data.columns

Index(['safe_loans', 'grade = A', 'grade = B', 'grade = C', 'grade = D',
       'grade = E', 'grade = F', 'grade = G', 'term =  36 months',
       'term =  60 months', 'home_ownership = MORTGAGE',
       'home_ownership = OTHER', 'home_ownership = OWN',
       'home_ownership = RENT', 'emp_length = 1 year',
       'emp_length = 10+ years', 'emp_length = 2 years',
       'emp_length = 3 years', 'emp_length = 4 years', 'emp_length = 5 years',
       'emp_length = 6 years', 'emp_length = 7 years', 'emp_length = 8 years',
       'emp_length = 9 years', 'emp_length = < 1 year', 'emp_length = N/A'],
      dtype='object')

In [81]:
train_features = list(loans.columns)
train_features.remove(target)

In [82]:
my_decision_tree_new = decision_tree_create(train_data, train_features, 'safe_loans',
                                            0,
                                           max_depth=max_depth, 
                                            min_node_size=min_node_size,
                                            min_error_reduction=min_error_reduction)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Best feature: emp_length = N/A, best error: 0.36
Split on feature emp_length = N/A. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Early stopping condition 2 reached. Reached minimum node size.
----------------------

In [83]:
my_decision_tree_old = decision_tree_create(train_data,
                                            train_features,
                                            'safe_loans',
                                            current_depth=0,
                                            min_node_size = 0,
                                            max_depth = 6,
                                            min_error_reduction=-1
                                           )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Split on feature grade = B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Best feature: grade = C, best error: 0.33
Split on feature grade = C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Best feature: grade = D, best error: 0.30
Split on feature grade = D. (3826, 2058)
----------------------------

### Making predictions

In [84]:
print(validation_data.iloc[0])
print('Predicted class: %s ' % classify(my_decision_tree_new, validation_data.iloc[0]))


safe_loans                  -1
grade = A                    0
grade = B                    0
grade = C                    0
grade = D                    1
grade = E                    0
grade = F                    0
grade = G                    0
term =  36 months            0
term =  60 months            1
home_ownership = MORTGAGE    0
home_ownership = OTHER       0
home_ownership = OWN         0
home_ownership = RENT        1
emp_length = 1 year          0
emp_length = 10+ years       0
emp_length = 2 years         1
emp_length = 3 years         0
emp_length = 4 years         0
emp_length = 5 years         0
emp_length = 6 years         0
emp_length = 7 years         0
emp_length = 8 years         0
emp_length = 9 years         0
emp_length = < 1 year        0
emp_length = N/A             0
Name: 24, dtype: int64
Predicted class: -1 


In [85]:
classify(my_decision_tree_new, validation_data.iloc[0], annotate = True)

Split on term =  36 months = 0
Split on grade = A = 0
At leaf, predicting -1


-1

In [86]:
classify(my_decision_tree_old, validation_data.iloc[0], annotate = True)

Split on term =  36 months = 0
Split on grade = A = 0
Split on grade = B = 0
Split on grade = C = 0
Split on grade = D = 1
At leaf, predicting -1


-1

### Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for validation_set[0] shorter, longer, or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?



shorter

### Quiz question: For my_decision_tree_new trained with max_depth = 6, min_node_size = 100, min_error_reduction=0.0, is the prediction path for any point always shorter, always longer, always the same, shorter or the same, or longer or the same as for my_decision_tree_old that ignored the early stopping conditions 2 and 3?



shorter or the same

## Evaluating the model


In [87]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    
    # Once you've made the predictions, calculate the classification error and return it
    ## YOUR CODE HERE
    total_error = (prediction != data[target]).sum()
    error_ratio = total_error * 1. / len(data)
    return error_ratio
    

In [88]:
evaluate_classification_error(my_decision_tree_new, validation_data)

0.38367083153813014

In [89]:
evaluate_classification_error(my_decision_tree_old, validation_data)

0.3837785437311504

### Quiz question: Is the validation error of the new decision tree (using early stopping conditions 2 and 3) lower than, higher than, or the same as that of the old decision tree from the previous assignment?



(slightly) lower

### Exploring the effect of `max_depth`


In [90]:
model_1 = decision_tree_create(train_data,
                                            train_features,
                                            'safe_loans',
                                            current_depth=0,
                                            min_node_size = 0,
                                            max_depth = 2,
                                            min_error_reduction=-1
                                           )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Early stopping condition 1 reached. Reached maximum depth
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Early stopping condition 1 reached. Reached maximum depth
--------------------------------------------------------------------
Subtree, depth = 1 (28001 data points).
Best feature: grade = D, best error: 0.42
Split on feature grade = D. (23300, 4701)
--------------------------------------------------------------------
Subtree,

In [91]:
model_2 = decision_tree_create(train_data,
                                            train_features,
                                            'safe_loans',
                                            current_depth=0,
                                            min_node_size = 0,
                                            max_depth = 6,
                                            min_error_reduction=-1
                                           )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Split on feature grade = B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Best feature: grade = C, best error: 0.33
Split on feature grade = C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Best feature: grade = D, best error: 0.30
Split on feature grade = D. (3826, 2058)
----------------------------

In [92]:
model_3 = decision_tree_create(train_data,
                                            train_features,
                                            'safe_loans',
                                            current_depth=0,
                                            min_node_size = 0,
                                            max_depth = 14,
                                            min_error_reduction=-1
                                           )

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Split on feature grade = B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Best feature: grade = C, best error: 0.33
Split on feature grade = C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Best feature: grade = D, best error: 0.30
Split on feature grade = D. (3826, 2058)
----------------------------

Best feature: emp_length = 5 years, best error: 0.46
Split on feature emp_length = 5 years. (2335, 227)
--------------------------------------------------------------------
Subtree, depth = 12 (2335 data points).
Best feature: grade = C, best error: 0.46
Split on feature grade = C. (0, 2335)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 12 (227 data points).
Best feature: grade = C, best error: 0.50
Split on feature grade = C. (0, 227)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 11 (230 data points).
Best feature: grade = C, best error: 0.50
Split on feature grade = C. (0, 230)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 9 (8917 data points).
Best feature: grade = C, best error: 0.40
Split on feature grade = C. (8917, 0)
Creating leaf node.
-----------------------------------------------------------------

In [93]:
print ("Training data, classification error (model 1):", evaluate_classification_error(model_1, train_data))
print ("Training data, classification error (model 2):", evaluate_classification_error(model_2, train_data))
print ("Training data, classification error (model 3):", evaluate_classification_error(model_3, train_data))

Training data, classification error (model 1): 0.40003761014399314
Training data, classification error (model 2): 0.38185041908446166
Training data, classification error (model 3): 0.3761820330969267


In [94]:
print ("Validation data, classification error (model 1):", evaluate_classification_error(model_1, validation_data))
print ("Validation data, classification error (model 2):", evaluate_classification_error(model_2, validation_data))
print ("Validation data, classification error (model 3):", evaluate_classification_error(model_3, validation_data))

Validation data, classification error (model 1): 0.3981042654028436
Validation data, classification error (model 2): 0.3837785437311504
Validation data, classification error (model 3): 0.37731581214993537


### Quiz question: Which tree has the largest complexity?



model_3


### Quiz question: Is it always true that the most complex tree will result in the lowest classification error in the validation_set?


false

## Exploring the effect of `min_error`

In [95]:
model_4 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=0,
                                max_depth = 6,
                                min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Split on feature grade = B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Best feature: grade = C, best error: 0.33
Split on feature grade = C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Best feature: grade = D, best error: 0.30
Split on feature grade = D. (3826, 2058)
----------------------------

In [96]:
model_5 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=0,
                                max_depth = 6,
                                min_error_reduction=0.0)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Early stopping condition 3 reached. Minimum error reduction.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Best feature: emp_length = N/A, best error: 0.36
Split on feature emp_length = N/A. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points).
Best feature: emp_length = < 1 year, best error: 0.35
Split on feature emp_length = <

In [97]:
model_6 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=0,
                                max_depth = 6,
                                min_error_reduction=5)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Early stopping condition 3 reached. Minimum error reduction.


In [98]:
print ("Validation data, classification error (model 4):", evaluate_classification_error(model_4, validation_data))
print ("Validation data, classification error (model 5):", evaluate_classification_error(model_5, validation_data))
print ("Validation data, classification error (model 6):", evaluate_classification_error(model_6, validation_data))

Validation data, classification error (model 4): 0.3837785437311504
Validation data, classification error (model 5): 0.3844248168892719
Validation data, classification error (model 6): 0.496553209823352


In [99]:
def count_leaves(tree):
    if tree['is_leaf']:
        return 1
    return count_leaves(tree['left']) + count_leaves(tree['right'])

### Quiz Question: Using the complexity definition above, which model (model_4, model_5, or model_6) has the largest complexity? Did this match your expectation?



In [100]:
count_leaves_m4 = count_leaves(model_4)
count_leaves_m4

19

In [101]:
count_leaves_m5 = count_leaves(model_5)
count_leaves_m5

13

In [102]:
count_leaves_m6 = count_leaves(model_6)
count_leaves_m6

1

model 4 has the largest complexity. Match my expectation

### Quiz Question: model_4 and model_5 have similar classification error on the validation set but model_5 has lower complexity? Should you pick model_5 over model_4?



yes

## Exploring the effect of `min_node_size`


In [103]:
model_7 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=0,
                                max_depth = 6,
                                min_error_reduction=-1)
model_8 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=2000,
                                max_depth = 6,
                                min_error_reduction=-1)
model_9 = decision_tree_create(train_data,
                                train_features,
                                'safe_loans',
                                current_depth=0,
                                min_node_size=50000,
                                max_depth = 6,
                                min_error_reduction=-1)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Best feature: term =  36 months, best error: 0.42
Split on feature term =  36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Best feature: grade = A, best error: 0.35
Split on feature grade = A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Best feature: grade = B, best error: 0.35
Split on feature grade = B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Best feature: grade = C, best error: 0.33
Split on feature grade = C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Best feature: grade = D, best error: 0.30
Split on feature grade = D. (3826, 2058)
----------------------------

Best feature: grade = E, best error: 0.28
Split on feature grade = E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
Early stopping condition 1 reached. Reached maximum depth
--------------------------------------------------------------------
Subtree, depth = 6 (2133 data points).
Early stopping condition 1 reached. Reached maximum depth
--------------------------------------------------------------------
Subtree, depth = 5 (2058 data points).
Best feature: grade = E, best error: 0.35
Split on feature grade = E. (2058, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 4 (2190 data points).
Best feature: grade = D, best error: 0.42
Split on feature grade = D. (2190, 0)
Creating leaf node.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Early stopping condition 2 reached. Reached minimum node size

In [104]:
print ("Validation data, classification error (model 7):", evaluate_classification_error(model_7, validation_data))
print ("Validation data, classification error (model 8):", evaluate_classification_error(model_8, validation_data))
print ("Validation data, classification error (model 9):", evaluate_classification_error(model_9, validation_data))

Validation data, classification error (model 7): 0.3837785437311504
Validation data, classification error (model 8): 0.38453252908229213
Validation data, classification error (model 9): 0.503446790176648


### Quiz Question: Using the results obtained in this section, which model (model_7, model_8, or model_9) would you choose to use?



model 8 (same error as model 7 but less complex)